In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import scanpy.api as sc
from igraph import *
from MulticoreTSNE import MulticoreTSNE as TSNE #faster TSNE alternative
from anndata import read_h5ad
import dill # for saving everything

sc.logging.print_versions()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


scanpy==1.3.1 anndata==0.6.10 numpy==1.14.3 scipy==1.1.0 pandas==0.23.0 scikit-learn==0.19.1 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


# Load data

## 18, 21 and 24 months

In [ ]:
path = '/data/maca/data/'
maca10x182124metadata = pd.read_csv('/data/maca/data/MACA_10x_18-21-24_qc.csv',usecols = ['channel','tissue','tissue.notes','mouse.age','mouse.sex'])

maca10x182124metadata = maca10x182124metadata.rename(columns = {'mouse.age':'age','mouse.sex':'sex','tissue.notes':'subtissue'})
maca10x182124metadata.index = range(len(maca10x182124metadata))

In [ ]:
maca10x182124metadata.head()
#len(maca10x182124metadata)

In [ ]:
adata182124aux = []
for i in range(0,len(maca10x182124metadata)):
    path = '/data/maca/data/' + maca10x182124metadata.channel[i] + '/'
    foo = sc.read(path + 'matrix.mtx', cache=True).transpose()
    foo.var_names = np.genfromtxt(path + 'genes.tsv', dtype=str)[:, 1]
    foo.obs_names = np.genfromtxt(path + 'barcodes.tsv', dtype=str)
    foo.obs['tissue'] = str(maca10x182124metadata.tissue[i]).title()
    #foo.obs['subTissue'] = str(maca10x182124metadata.subtissue[i]).title()
    
    
    if pd.isnull(maca10x182124metadata.subtissue[i]):
        foo.obs['subtissue'] = maca10x182124metadata.subtissue[i]
    else:        
        foo.obs['subtissue'] = str(maca10x182124metadata.subtissue[i]).title()
        
    
    foo.obs['age'] = str(maca10x182124metadata.age[i]) + 'm'
    
    if maca10x182124metadata.sex[i] == 'M':
        foo.obs['sex'] = 'male'
    else:
        foo.obs['sex'] = 'female'
    
    if maca10x182124metadata.tissue[i] == 'MUSCLE':
        foo.obs['tissue'] = 'Limb_Muscle'
        
    sc.pp.filter_cells(foo, min_genes=250)
    
    foo.obs['channel'] = maca10x182124metadata.channel[i]# + foo.obs_names.split("-")[0]
    
    foo.obs['cell_ontology_class'] = 'NA' #maca10x3metadata.cell_ontology_class[i]
    #foo.obs['CellTypeID'] = maca10x3metadata.cell_ontology_id[i]
    #foo.obs['FreeAnn'] = maca10x3metadata.free_annotation[i]
    
    adata182124aux.append(foo)
    #print(adata3.X.size*8/1000000) # convert to Mb


first = adata182124aux.pop()
adata182124 = first.concatenate(adata182124aux)
adata182124.obs['cell'] = adata182124.obs['channel'] + '_' + adata182124.obs_names
adata182124.obs['cell'] = adata182124.obs['cell'].apply(lambda x: pd.Series(x.split('-')))[0]
adata182124.obs['method'] = 'droplet'

adata182124.obs.head()

In [ ]:
set(adata182124.obs['tissue'])


## 3months

In [ ]:
path = '/data/maca/data/10x/'
maca10x3metadata = pd.read_csv('/data/maca/data/10x/MACA_10x_3_qc.csv',usecols = ['channel','tissue','tissue.notes','mouse.age','mouse.sex','pool'])
maca10x3metadata2 = pd.read_csv('/data/maca/data/10x/TM_droplet_metadata.csv', low_memory=False)

maca10x3metadata = maca10x3metadata.rename(columns = {'mouse.age':'age','mouse.sex':'sex','tissue.notes':'subtissue'})
#maca10x3metadata = maca10x3metadata[maca10x3metadata.age == 3]
maca10x3metadata.index = range(len(maca10x3metadata))

maca10x3metadata.head()

In [ ]:
adata3aux = []
for i in range(0,len(maca10x3metadata)):
    #print(i)
    path = '/data/maca/data/10x/' + maca10x3metadata.channel[i] + '/'
    foo = sc.read(path + 'matrix.mtx', cache=True).transpose()
    foo.var_names = np.genfromtxt(path + 'genes.tsv', dtype=str)[:, 1]
    foo.obs_names = np.genfromtxt(path + 'barcodes.tsv', dtype=str)
    foo.obs['tissue'] = maca10x3metadata.tissue[i]
    foo.obs['subtissue'] = maca10x3metadata.subtissue[i]
    foo.obs['age'] = str(maca10x3metadata.age[i]) + 'm'
    
    if maca10x3metadata.sex[i] == 'M':
        foo.obs['sex'] = 'male'
    else:
        foo.obs['sex'] = 'female'
    
    if maca10x3metadata.tissue[i] == 'Muscle':
        foo.obs['tissue'] = 'Limb_Muscle'
        
    sc.pp.filter_cells(foo, min_genes=250)
    
    foo.obs['channel'] = maca10x3metadata.channel[i]# + foo.obs_names.split("-")[0]
    
    foo.obs['cell_ontology_class'] = 'NA' #maca10x3metadata.cell_ontology_class[i]
    #foo.obs['CellTypeID'] = maca10x3metadata.cell_ontology_id[i]
    #foo.obs['FreeAnn'] = maca10x3metadata.free_annotation[i]
    
    adata3aux.append(foo)
    #print(adata3.X.size*8/1000000) # convert to Mb

first = adata3aux.pop()
adata3 = first.concatenate(adata3aux)
adata3.obs['cell'] = adata3.obs['channel'] + '_' + adata3.obs_names
adata3.obs['cell'] = adata3.obs['cell'].apply(lambda x: pd.Series(x.split('-')))[0]
adata3.obs['method'] = 'droplet'
display(adata3.obs.head())
adata3

In [ ]:
set(adata3.obs['tissue'])

## concatenate all time points

In [ ]:
adata = adata182124.concatenate([adata3])
adata

## import tabula-muris facs data

In [ ]:
# tm_facs_metadata = pd.read_csv('/data/maca/data/facs3mo/TM_facs_metadata.csv',usecols = ['cell','tissue','subtissue','mouse.sex','method','cell_ontology_class','cell_ontology_id','free_annotation'])
# tm_facs_metadata = tm_facs_metadata.rename(columns = {'mouse.sex':'sex'})
# display(tm_facs_metadata.head())
# tm_facs_metadata = tm_facs_metadata.set_index('cell')
# tm_facs_metadata.tail()

In [ ]:
# tm_facs_data = read_h5ad('/data/maca/data/facs3mo/TM_facs_mat.h5ad')
# tm_facs_data

In [ ]:
# tm_facs_data.obs.tail()

In [ ]:
# #tm_facs_data.obs['cell_ontology_class'] = tm_facs_data.obs['index'].map(tm_facs_metadata['cell_ontology_class'])
# tm_facs_data.obs = tm_facs_metadata
# tm_facs_data.obs['age'] = '3m'
# for i in range(0,len(tm_facs_data.obs)):
#     if tm_facs_data.obs['sex'][i] == 'M':
#         tm_facs_data.obs['sex'][i] = 'male'
#     else:
#         tm_facs_data.obs['sex'][i] = 'female'
#     if tm_facs_data.obs['tissue'][i] == 'Mammary_Gland':
#         tm_facs_data.obs['tissue'][i] = 'Mammary'

# tm_facs_data.obs.head()

In [ ]:
# tm_facs_data

In [ ]:
# adata = adata.concatenate(tm_facs_data)
# adata

In [ ]:
adata.write('./write/maca-droplet.h5ad')

In [ ]:
import gc
gc.collect()

# Import all data

In [ ]:
adata = read_h5ad('./write/maca-droplet.h5ad')
type(adata.X)


In [ ]:
adata.shape

In [ ]:
adata

In [ ]:
adata.obs.head()

# Preprocessing

In [ ]:
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.filter_cells(adata, min_genes=250)
adata

In [ ]:
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = np.sum(adata.X, axis=1).A1
adata

In [ ]:
axs = sc.pl.violin(adata, ['n_genes', 'n_counts'],
                   jitter=0.4, multi_panel=True)

In [ ]:
ax = sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4) #simple lib size normalization?
adata.raw = adata

In [ ]:
filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.0125, max_mean=10, min_disp=0.5)
sc.pl.filter_genes_dispersion(filter_result)

In [ ]:
adata = adata[:, filter_result.gene_subset]
adata.shape

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.scale(adata, max_value=10, zero_center=False)

# Exploration
## PCA

In [ ]:
sc.tl.pca(adata)

In [ ]:
ax = sc.pl.pca_scatter(adata, color=['tissue'], right_margin=0.5, save='_all_ages_droplet_tissue.pdf')

In [ ]:
ax = sc.pl.pca_scatter(adata, color=['age'], right_margin=0.5, save='_all_ages_droplet_age.pdf')

In [ ]:
ax = sc.pl.pca_scatter(adata, color=['sex'], right_margin=0.5, save='_all_ages_droplet_sex.pdf')

In [ ]:
ax = sc.pl.pca_scatter(adata, color='n_counts', right_margin=0.5, save='_all_ages_droplet_counts.pdf')

In [ ]:
ax = sc.pl.pca_scatter(adata, color='method', right_margin=0.5, save='_all_ages_droplet_method.pdf')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

## Louvain clustering

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15)#, method='gauss')
sc.tl.louvain(adata, resolution = 0.3)

In [ ]:
adata

## UMAP

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(adata, color=['tissue'], right_margin=0.5, save='_all_ages_droplet_tissue.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(adata, color=['age'], right_margin=0.5, save='_all_ages_droplet_age.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(adata, color=['sex'], right_margin=0.5, save='_all_ages_droplet_sex.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(adata, color=['louvain'], right_margin=0.5, save='_all_ages_droplet_louvain.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(adata, color=['method'], right_margin=0.5, save='_all_ages_droplet_method.pdf')

## tSNE

In [ ]:
sc.tl.tsne(adata, perplexity=50)

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.tsne(adata, color=['tissue'], right_margin=0.5, save='_all_ages_droplet_tissue.pdf')

In [ ]:
sc.pl.tsne(adata, color=['age'], right_margin=0.5, save='_all_ages_droplet_age.pdf')

In [ ]:
sc.pl.tsne(adata, color=['sex'], right_margin=0.5, save='_all_ages_droplet_sex.pdf')

In [ ]:
sc.pl.tsne(adata, color=['method'], right_margin=0.5, save='_all_ages_droplet_method.pdf')

In [ ]:
sc.pl.tsne(adata, color=['louvain'], right_margin=0.5, save='_all_ages_droplet_louvain.pdf')

# Save processed data

In [ ]:
adata.write('./write/maca-droplet.processed.h5ad')

# Explore & play 

In [ ]:
inflammation_markers_all = ['A2m','Abr','Ace','Ackr1','Ackr2','Acod1','Acp5','Ada','Adam8','Adam17',
                        'Adamts12','Adcyap1','Adipoq','Adora1','Adora2a','Adora2b','Adora3',
                        'Adra2a','Adrb2','Afap1l2','Ager','Agt','Agtr1a','Agtr1b','Agtr2','Ahcy',
                        'Ahsg','Aim2','Aimp1','Ak7','Akt1','Alox5','Alox5ap','Ankrd42','Ano6',
                        'Anxa1','Aoah','Aoc3','Apoa1','Apod','Apoe','Ash1l','Atrn','Axl']

inflammation_markers = list()
for i in range(0,len(inflammation_markers_all)):
    if inflammation_markers_all[i] in list(adata.var_names):
#         inflammation_markers = 
        inflammation_markers.append(inflammation_markers_all[i])

inflammation_markers

In [ ]:
#sc.pl.dotplot(adata[adata.obs['age']=='3m'], ['Cdkn2a','Cd4','Mmp12','Fabp4','Timp1','Il6'], groupby='tissue', log=False)
# sc.pl.dotplot(adata[adata.obs['age']=='3m'], inflammation_markers, groupby='tissue', log=False)
# sc.pl.dotplot(adata[adata.obs['age']=='18m'], inflammation_markers, groupby='tissue', log=False)
# sc.pl.dotplot(adata[adata.obs['age']=='21m'], inflammation_markers, groupby='tissue', log=False)
sc.pl.dotplot(adata[adata.obs['age']=='24m'], inflammation_markers, groupby='tissue', log=False)

In [ ]:
sc.pl.stacked_violin(adata, inflammation_markers, groupby='tissue')

In [ ]:
sc.pl.heatmap(adata, inflammation_markers, groupby='tissue')

In [ ]:
tissues_available = list(set(adata.obs['tissue']))
tissues_available.sort()
pd.DataFrame(tissues_available,columns=['Tissues'])
tissofinterest = tissues_available[6]
tiss = adata[adata.obs['tissue'] == tissofinterest,:]
tissofinterest

In [ ]:
sc.pp.neighbors(tiss, n_neighbors=15)#, method='gauss')
sc.tl.louvain(tiss, resolution = 0.2)

In [ ]:
sc.tl.paga(tiss,groups="age")

In [ ]:
sc.pl.paga(tiss)#, color = 'age')

In [ ]:
sc.pl.paga(tiss, color='age')

In [ ]:
adata = read_h5ad('./write/maca-droplet.processed.h5ad')

In [ ]:
set(adata.obs['method'])

In [ ]:
sc.tl.rank_genes_groups(adata, 'age', groups=['24m'], reference='3m')

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=20, save='_24m_vs_3m_all_tissues.pdf')

In [ ]:
cluster_genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
cluster_genes.head(10)

In [ ]:
from goatools import obo_parser
import wget
import os
import sys
import pandas as pd
import mygene
# from unpack_dict_to_col_pd import unpack
mg = mygene.MyGeneInfo()

In [ ]:
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = './write'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.mkdir(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'
    
print(go_obo)

In [ ]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

In [ ]:
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()

In [ ]:
from goatools.associations import read_gaf

In [ ]:
from goatools.obo_parser import GODag

obodag = GODag("go-basic.obo")

In [ ]:
from __future__ import print_function
from goatools.associations import read_ncbi_gene2go

geneid2gos_mouse = read_ncbi_gene2go("gene2go", taxids=[10090])

print("{N:,} annotated mouse genes".format(N=len(geneid2gos_mouse)))

In [ ]:
from goatools.test_data.genes_NCBI_10090_ProteinCoding import GeneID2nt as GeneID2nt_mus

In [ ]:
from goatools.go_enrichment import GOEnrichmentStudy

goeaobj = GOEnrichmentStudy(
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        geneid2gos_mouse, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method

In [ ]:
#read in gene symbols:
ephys_genes = cluster_genes['24m'] #pd.read_csv('/Volumes/Brain2017/project/GO_term_analysis_ephys/top_ephys/top_ephys_genes_fine.csv', index_col = 0)

#remove r squared columns:
# cols = [c for c in ephys_genes.columns if c.lower()[:2] != 'r_']
ephys_symbol = ephys_genes#[cols]
ephys_symbol = pd.DataFrame(ephys_symbol)

#convert gene symbols to entrezgene ids, remove NAs and save to dictionary for each for each ephys measure
output = {}
for c in ephys_symbol.columns:
    out = mg.querymany(ephys_symbol[c], scopes='symbol', fields='entrezgene', species='mouse')
    out = pd.DataFrame(out)
    out = out.dropna(subset = ['entrezgene'])
    out.entrezgene = out.entrezgene.astype(int)
    entrezgene = out.entrezgene
    #filter for protein coding genes:
    entrezgene = entrezgene[entrezgene.isin(GeneID2nt_mus.keys())].dropna()
    output[c] = entrezgene
    


In [ ]:
out = mg.querymany(ephys_genes, scopes='symbol', fields='entrezgene', species='mouse')
df = pd.DataFrame(out,columns=['query','_id','_score','entrezgene'])
# df = df.loc[:,['query','entrezgene']]
geneid2symbol = {}
for r in range(df.shape[0]):
    if df['_id'][r] == df['entrezgene'][r]:
        geneid2symbol[int(df['entrezgene'][r])] = df['query'][r]

In [ ]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
goea_results_sig_dict = {}
for key in output.keys():
    # 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
    #geneids_study = geneid2symbol.keys()
    geneids_study = output[key]
    goea_results_all = goeaobj.run_study(geneids_study)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    goeaobj.wr_xlsx('./write/' + 'all_tissues_droplet' + '_' + str(key) + '_vs_3m_' + "GOEA_resultsFine_sig.xlsx", goea_results_sig)    

    goea_results_sig_dict[key] = goea_results_sig

In [ ]:
go_names = [r.name for r in goea_results_sig]
print(len(go_names)) # Includes ONLY signficant results

In [ ]:
goid2goobj_all = {nt.GO:nt.goterm for nt in goea_results_all}
print(len(goid2goobj_all))

In [ ]:
import collections as cx
word2cnt = cx.Counter([word for name in go_names for word in name.split()])
print(word2cnt.most_common(10))

In [ ]:
word2cnt

In [ ]:
freq_seen = ['RNA', 'translation', 'mitochond', 'ribosomal', 'ribosome',
             'adhesion', 'endoplasmic', 'nucleotide', 'apoptotic','apoptosis',
             'inflammation','immune','MHC','senescence','aging','necrose',
             'growth','differentiation', 'development']

# freq_seen = ['adhesion','apoptotic','immune','MHC','growth','differentiation']

# Collect significant GOs for words in freq_seen (unordered)
word2siggos = cx.defaultdict(set)
# Loop through manually curated words of interest
for word in freq_seen:
    # Check each significant GOEA result for the word of interest
    for rec in goea_results_sig:
        if word in rec.name:
            word2siggos[word].add(rec.GO)
# Sort word2gos to have the same order as words in freq_seen
word2siggos = cx.OrderedDict([(w, word2siggos[w]) for w in freq_seen])

In [ ]:
# Plot set of GOs for each frequently seen word
from goatools.godag_plot import plot_goid2goobj
for word, gos in word2siggos.items():
    goid2goobj = {go:goid2goobj_all[go] for go in gos}
    plot_goid2goobj(
        "./figures/go_terms_24m_vs_3m_word_{WORD}.png".format(WORD=word),
        goid2goobj, # source GOs to plot and their GOTerm object
        study_items=15, # Max number of gene symbols to print in each GO term
        id2symbol=geneid2symbol, # Contains GeneID-to-Symbol from Step 1
        goea_results=goea_results_all, # pvals used for GO Term coloring
        dpi=150)

In [ ]:
fout = "./figures/GO_24m_vs_3m_word_genes.txt"
go2res = {nt.GO:nt for nt in goea_results_all}
with open(fout, "w") as prt:
#     prt.write("""This file is generated by test_nbt3102.py and is intended to confirm
# this statement in the GOATOOLS manuscript:

#         We observed:
#             93 genes associated with RNA,
#             47 genes associated with translation,
#             70 genes associated with mitochondrial or mitochondrian, and
#             37 genes associated with ribosomal, as reported by GOATOOLS.

# """)
    for word, gos in word2siggos.items():
        # Sort first by BP, MF, CC. Sort second by GO id.
        gos = sorted(gos, key=lambda go: [go2res[go].NS, go])
        genes = set()
        for go in gos:
            genes |= go2res[go].study_items
        genes = sorted([geneid2symbol[g] for g in genes])
        prt.write("\n{WD}: {N} study genes, {M} GOs\n".format(WD=word, N=len(genes), M=len(gos)))
        prt.write("{WD} GOs: {GOs}\n".format(WD=word, GOs=", ".join(gos)))
        for i, go in enumerate(gos):
            res = go2res[go]
            prt.write("{I}) {NS} {GO} {NAME} ({N} genes)\n".format(
                I=i, NS=res.NS, GO=go, NAME=res.name, N=res.study_count))
        prt.write("{N} study genes:\n".format(N=len(genes)))
        N = 10 # Number of genes per line
        mult = [genes[i:i+N] for i in range(0, len(genes), N)]
        prt.write("  {}\n".format("\n  ".join([", ".join(str(g) for g in sl) for sl in mult])))
    print("  WROTE: {F}\n".format(F=fout))

In [ ]:
import gseapy as gp

In [ ]:
names = list(gp.get_library_name())
names

In [ ]:
# run enrichr
# if you are only intrested in dataframe that enrichr returned, please set no_plot=True

# list, dataframe, series inputs are supported
enr = gp.enrichr(gene_list=cluster_genes['24m'],
                 # or gene_list='./gene_list.txt', or gene_list=glist
                 description='24m vs 3m kegg',
                 gene_sets='BioCarta_2016',
                 outdir='./write/enrichr_' + '24m_vs_3m' + 'bioC',
                 cutoff=0.5 # test dataset, use lower value of range(0,1)
                )

enr.res2d.head()

In [ ]:
fig = gp.plot.barplot(enr.res2d, cutoff=0.05, figsize=(6.5, 6), top_term=10, color='salmon', title='24m vs 3m')
fig.savefig('figures/enrichr_' + '24m_vs_3m' + 'bioC', bbox_inches='tight')

### GSVA

In [ ]:
import pandas as pd
from GSVA import gsva, gmt_to_dataframe
# Some extras to look at the high dimensional data
from plotnine import *
from sklearn.manifold import TSNE

In [ ]:
genesets_df = gmt_to_dataframe('c2.all.v6.2.symbols.gmt')
genesets_df.head()

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
expression_df = pd.DataFrame({group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores']})#.head(5)

In [ ]:
expression_df.set_index('24m_n')
expression_df.columns = ['symbol','z-score']

In [ ]:
XV = TSNE(n_components=2).\
    fit_transform(expression_df.T)
df = pd.DataFrame(XV).rename(columns={0:'x',1:'y'})
(ggplot(df,aes(x='x',y='y'))
 + geom_point(alpha=0.2)
)

In [ ]:
pathways_df = gsva(expression_df,genesets_df)
# pathways_df.iloc[0:5,0:5]
